# Create location lookup

> Fill in a module description here

In [1]:
#| default_exp create_location_lookup

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import warnings
import pandas as pd
from pandas.errors import DtypeWarning
from pathlib import Path
import json
import numpy as np
import time
import redis
from geopy.geocoders import Nominatim
from openai import OpenAI
import math
import geopy
import googlemaps
import os

from data_preprocessor import const

Load lookup table if it already exists

In [5]:
#|exports
lon_lat_lookup_path = Path(const.output_path, "lon_lat_lookup.json")
if lon_lat_lookup_path.is_file():
    with open(lon_lat_lookup_path, "r") as f:
        place_lon_lat_lookup = json.loads(f.read())
else:
    place_lon_lat_lookup = {}

Create a lookup table of UK cities and towns to lon-lat

In [6]:
#|exports

df_worldcities = pd.read_csv(Path(const.data_path, 'worldcities.csv'))

# Filter to UK
df_worldcities = df_worldcities[df_worldcities['country'] == 'United Kingdom']

# Add cities

for i, row in df_worldcities.iterrows():
    place = row['city'].strip().lower()
    place_lon_lat_lookup[place] = { 'lat' : row['lat'], 'lng' : row['lng'] }
    
# Add administration areas, by averaging

admin_lon_lats = {}
for i, row in df_worldcities.iterrows():
    if pd.isna(row['admin_name']):
        continue

    if (row['city'].strip().lower() == row['admin_name'].strip().lower()) or \
        (row['admin_name'].strip().lower() == f"{row['city'].strip()}, City of".strip().lower()):
        place = row['city']
    else:
        place = f"{row['city'].strip()}, {row['admin_name'].strip()}".strip().lower()
    
    if place in place_lon_lat_lookup:
        continue
    if not place in admin_lon_lats:
        admin_lon_lats[place] = []
    admin_lon_lats[place].append( (row['lat'], row['lng']) )
    
# average the administration areas

for k,vs in admin_lon_lats.items():
    place_lon_lat_lookup[k] = {
        'lat' : sum(map(lambda e: e[0], vs)) / len(vs),
        'lng' : sum(map(lambda e: e[1], vs))  / len(vs),
    }

Convert locations in the scraped sponsor data to lon-lats as well

In [7]:
#|exports
df_sponsors = pd.read_csv(Path(const.data_path, "all-skilled-home-care-sponsors.csv"))
df_sponsors.head()

,Organisation Name,Address 1,Address 2,Postcode,Website,CQC_URL,Town/City,County,Type & Rating,Route,First appeared,Last appeared,Provider name,Status,Local authority,NHS
0,First in Care Services Ltd,"Kingsgate, Office 623, Regus",62 High Street,RH1 1SG,https://www.firstincare.co.uk,https://www.cqc.org.uk/location/1-9817102215,Redhill,NaN,Worker (A rating),Skilled Worker,2023-03-06,2024-05-15,First In Care Services Ltd,Active,False,False
1,Midshires Care Limited,6 College Road,NaN,HA1 1BE,https://www.helpinghands.co.uk,https://www.cqc.org.uk/location/1-3045672865,Alcester,Warwickshire,Worker (A rating),Skilled Worker,2022-01-12,2024-05-15,Midshires Care Limited,Active,False,False
2,West Sussex Care Services Limited,Units 3 - 4,"20 Northbrook Trading Estate, Northbrook Road",BN14 8PN,NaN,https://www.cqc.org.uk/location/1-9329666383,Worthing,West Sussex,Worker (A rating),Skilled Worker,2022-10-17,2024-05-15,West Sussex Care Services Limited,Active,False,False
3,Kitec Healthcare Services Limited,"Office B, Elstow Road",Kempston,MK42 9QZ,NaN,https://www.cqc.org.uk/location/1-16437635737,Bedford,NaN,Worker (A rating),Skilled Worker,2022-04-12,2024-05-15,Kitec Healthcare Services Limited,Active,False,False
4,JSS Homecare Ltd,"Yeovil Innovation Centre, Copse Road","Lufton Trading Estate, Lufton",BA22 8RN,NaN,https://www.cqc.org.uk/location/1-11484415015,Yeovil,England,Worker (A rating),Skilled Worker,2023-06-15,2024-05-15,JSS Homecare Ltd,Active,False,False


In [8]:
#|exports
client = OpenAI()

def correct_placename(placename):
    system_context = (
        "Given a mispelled version of a UK placename, you output either of the following: "
        "If you think it is a mispelled version of a UK placename, you output the corrected version. "
        "If you think string does not refer to a place, you output 'INVALID'."
    )
   
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_context},
            {"role": "user", "content": placename},
        ]
    )
    
    return response.choices[0].message.content

In [9]:
correct_placename("Stonehouse, Glos")

'Stonehouse, Gloucestershire'

In [10]:
#|exports
places_corrections_path = Path(const.output_path, "places_corrections.json")
if places_corrections_path.is_file():
    with open(places_corrections_path, "r") as f:
        places_corrections = json.loads(f.read())
else:
    places_corrections = {}

In [11]:
#|exports
df_sponsors = pd.read_csv(Path(const.data_path, "all-skilled-home-care-sponsors.csv"))
df_sponsors['Town/City'] = df_sponsors['Town/City'].str.lower().str.strip()

places = []

for i, row in df_sponsors.iterrows():
    if pd.isna(row['County']):
        places.append(row['Town/City'].lower().strip())
    elif pd.isna(row['Town/City']):
        places.append(row['County'].lower().strip())
    else:
        places.append(f"{row['Town/City'].strip()}, {row['County'].strip()}".lower())
        
places = set(places)

In [12]:
#|exports
api_key = os.environ.get('GOOGLE_MAPS_API_KEY')
gmaps = googlemaps.Client(key=api_key)

places = places - set(place_lon_lat_lookup.keys())
places = places - set(places_corrections.keys())

for i, place in enumerate(places):
    print(f"{i+1}/{len(places)}", end='\r')
    
    success = False
    while not success:
        if place in place_lon_lat_lookup or place in places_corrections:
            continue

        result = gmaps.geocode(f"{place.title()}, UK")
        location = result[0]['geometry']['location']
        
        if not location:
            place_corrected = correct_placename(place).lower()
            result = gmaps.geocode(f"{place_corrected.title()}, UK")
            location = result[0]['geometry']['location']
            print(f"{place} -> {place_corrected}")
            if location: places_corrections[place] = place_corrected
        else:
            place_corrected = place
        
        if location:
            place_lon_lat_lookup[place_corrected] = { 'lat' : location['lat'], 'lng' : location['lng'] }
        else:
            print(f"  {place} still invalid")
            place_lon_lat_lookup[place_corrected] = "invalid"
            
        success = True

Get the lon-lats of all postcodes as well

In [13]:
#|exports
postcode_lon_lats_path = Path(const.output_path, "postcode_lon_lats.json")
if postcode_lon_lats_path.is_file():
    with open(postcode_lon_lats_path, "r") as f:
        postcode_lon_lats = json.loads(f.read())
else:
    postcode_lon_lats = {}

In [14]:
#|exports
api_key = os.environ.get('GOOGLE_MAPS_API_KEY')
gmaps = googlemaps.Client(key=api_key)

postcodes = set(df_sponsors['Postcode'])

for i, postcode in enumerate(postcodes):
    print(f"{i+1}/{len(postcodes)}", end='\r')
    
    if postcode in postcode_lon_lats:
        continue
    
    success = False

    result = gmaps.geocode(f"{postcode}, UK")
    location = result[0]['geometry']['location']
    
    if location:
        postcode_lon_lats[postcode] = location
    else:
        postcode_lon_lats[postcode] = -1

In [15]:
#|exports
"""
geolocator = Nominatim(user_agent="Alex Guli")

places = places - set(place_lon_lat_lookup.keys())
places = places - set(places_corrections.keys())

for i, place in enumerate(places):
    print(f"{i+1}/{len(places)}", end='\r')
    
    success = False
    while not success:
        try:
            if place in place_lon_lat_lookup or place in places_corrections:
                continue

            location = geolocator.geocode(f"{place.title()}, UK")
            
            if location is None:
                place_corrected = correct_placename(place).lower()
                location = geolocator.geocode(f"{place_corrected.title()}, UK")
                print(f"{place} -> {place_corrected}")
                if location is not None: places_corrections[place] = place_corrected
            else:
                place_corrected = place
            
            if location is not None:
                place_lon_lat_lookup[place_corrected] = { 'lat' : location.latitude, 'lng' : location.longitude }
            else:
                print(f"  {place} still invalid")
                place_lon_lat_lookup[place_corrected] = "invalid"
                
            success = True
        except geopy.exc.GeocoderUnavailable as e:
            print("\nFailed API")
            time.sleep(30)
"""

'\ngeolocator = Nominatim(user_agent="Alex Guli")\n\nplaces = places - set(place_lon_lat_lookup.keys())\nplaces = places - set(places_corrections.keys())\n\nfor i, place in enumerate(places):\n    print(f"{i+1}/{len(places)}", end=\'\r\')\n    \n    success = False\n    while not success:\n        try:\n            if place in place_lon_lat_lookup or place in places_corrections:\n                continue\n\n            location = geolocator.geocode(f"{place.title()}, UK")\n            \n            if location is None:\n                place_corrected = correct_placename(place).lower()\n                location = geolocator.geocode(f"{place_corrected.title()}, UK")\n                print(f"{place} -> {place_corrected}")\n                if location is not None: places_corrections[place] = place_corrected\n            else:\n                place_corrected = place\n            \n            if location is not None:\n                place_lon_lat_lookup[place_corrected] = { \'lat\' : loc

In [16]:
#|hide
# Will not use postcodes

"""

# Ignore DtypeWarning
warnings.filterwarnings('ignore', category=DtypeWarning)

# Add postcodes
r = redis.Redis()
pipe = r.pipeline()

path_dfs = list(Path(const.data_path, 'postcode_data').glob('*.csv'))
for i, path_df in enumerate(path_dfs):
    print(f"Processing postcode data ({i+1}/{len(path_dfs)})", end="\r")
    df_postcode = pd.read_csv(path_df)
    
    for j, row in df_postcode.iterrows():
        pipe.set(row['pcd'], f"{row['lat']}, {row['lng']}")
        place_lon_lat_lookup[row['pcd']] = { 'lat' : row['lat'], 'lng' : row['long'] }
        place_lon_lat_lookup[row['pcd2']] = { 'lat' : row['lat'], 'lng' : row['long'] }
        place_lon_lat_lookup[row['pcds']] = { 'lat' : row['lat'], 'lng' : row['long'] }

pipe.execute()
"""

'\n\n# Ignore DtypeWarning\nwarnings.filterwarnings(\'ignore\', category=DtypeWarning)\n\n# Add postcodes\nr = redis.Redis()\npipe = r.pipeline()\n\npath_dfs = list(Path(const.data_path, \'postcode_data\').glob(\'*.csv\'))\nfor i, path_df in enumerate(path_dfs):\n    print(f"Processing postcode data ({i+1}/{len(path_dfs)})", end="\r")\n    df_postcode = pd.read_csv(path_df)\n    \n    for j, row in df_postcode.iterrows():\n        pipe.set(row[\'pcd\'], f"{row[\'lat\']}, {row[\'lng\']}")\n        place_lon_lat_lookup[row[\'pcd\']] = { \'lat\' : row[\'lat\'], \'lng\' : row[\'long\'] }\n        place_lon_lat_lookup[row[\'pcd2\']] = { \'lat\' : row[\'lat\'], \'lng\' : row[\'long\'] }\n        place_lon_lat_lookup[row[\'pcds\']] = { \'lat\' : row[\'lat\'], \'lng\' : row[\'long\'] }\n\npipe.execute()\n'

Save lookup table

In [17]:
#|exports
with open(lon_lat_lookup_path, 'w') as f:
    json.dump(place_lon_lat_lookup, f)
    
with open(places_corrections_path, 'w') as f:
    json.dump(places_corrections, f)
    
with open(postcode_lon_lats_path, 'w') as f:
    json.dump(postcode_lon_lats, f)

In [18]:
#|exports
def lat_lng_to_mercator(lat, lng):
    r_major = 6378137.000
    x = r_major * math.radians(lng)
    scale = x/lng
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [19]:
#|exports
place_merc_lookup = { place : lat_lng_to_mercator(loc['lat'], loc['lng']) for place, loc in place_lon_lat_lookup.items() }    
place_merc_lookup_path = Path(const.output_path, "place_merc_lookup.json")
with open(place_merc_lookup_path, 'w') as f:
    json.dump(place_merc_lookup, f)

Update the sponsor data with location

In [20]:
df_sponsors['Town/City']

0              redhill
1             alcester
2             worthing
3              bedford
4               yeovil
             ...      
8759        werrington
8760       stourbridge
8761        castleford
8762         dronfield
8763    stoke-on-trent
Name: Town/City, Length: 8764, dtype: object

In [21]:
#|exports
df_sponsors = pd.read_csv(Path(const.data_path, "current-skilled-home-care-sponsors.csv"))
df_sponsors['Town/City'] = df_sponsors['Town/City'].str.lower().str.strip()
df_sponsors['Town/City'] = df_sponsors['Town/City'].replace(places_corrections)
df_sponsors.head()

,Organisation Name,Address 1,Address 2,Postcode,Website,CQC_URL,Town/City,County,Type & Rating,Route,First appeared,Last appeared,Provider name,Status
0,First in Care Services Ltd,"Kingsgate, Office 623, Regus",62 High Street,RH1 1SG,https://www.firstincare.co.uk,https://www.cqc.org.uk/location/1-9817102215,redhill,NaN,Worker (A rating),Skilled Worker,2023-03-06,2024-05-15,First In Care Services Ltd,Active
1,Midshires Care Limited,6 College Road,NaN,HA1 1BE,https://www.helpinghands.co.uk,https://www.cqc.org.uk/location/1-3045672865,alcester,Warwickshire,Worker (A rating),Skilled Worker,2022-01-12,2024-05-15,Midshires Care Limited,Active
2,West Sussex Care Services Limited,Units 3 - 4,"20 Northbrook Trading Estate, Northbrook Road",BN14 8PN,NaN,https://www.cqc.org.uk/location/1-9329666383,worthing,West Sussex,Worker (A rating),Skilled Worker,2022-10-17,2024-05-15,West Sussex Care Services Limited,Active
3,Kitec Healthcare Services Limited,"Office B, Elstow Road",Kempston,MK42 9QZ,NaN,https://www.cqc.org.uk/location/1-16437635737,bedford,NaN,Worker (A rating),Skilled Worker,2022-04-12,2024-05-15,Kitec Healthcare Services Limited,Active
4,JSS Homecare Ltd,"Yeovil Innovation Centre, Copse Road","Lufton Trading Estate, Lufton",BA22 8RN,NaN,https://www.cqc.org.uk/location/1-11484415015,yeovil,England,Worker (A rating),Skilled Worker,2023-06-15,2024-05-15,JSS Homecare Ltd,Active


In [22]:
#|exports
xs, ys = [], []

for i, row in df_sponsors.iterrows():
    if pd.isna(row['County']):
        place = row['Town/City'].lower().strip()
    elif pd.isna(row['Town/City']):
        place = row['County'].lower().strip()
    else:
        place = f"{row['Town/City'].strip()}, {row['County'].strip()}".lower()
    
    postcode = row['Postcode']
    
    if postcode in postcode_lon_lats and postcode_lon_lats[postcode] != -1:
        x, y = lat_lng_to_mercator(postcode_lon_lats[postcode]['lat'], postcode_lon_lats[postcode]['lng'])
        xs.append(x)
        ys.append(y)
    else:
        if place in place_lon_lat_lookup:
            x, y = place_merc_lookup[place]
            xs.append(x)
            ys.append(y)
        else:
            print(place)
            xs.append(-1)
            ys.append(-1)

In [24]:
#|exports
df_sponsors['merc_x'] = xs
df_sponsors['merc_y'] = ys

df_sponsors.to_csv(Path(const.output_path, "current-skilled-home-care-sponsors-with-loc.csv"), index=False)

In [25]:
df_sponsors.head()

,Organisation Name,Address 1,Address 2,Postcode,Website,CQC_URL,Town/City,County,Type & Rating,Route,First appeared,Last appeared,Provider name,Status,merc_x,merc_y
0,First in Care Services Ltd,"Kingsgate, Office 623, Regus",62 High Street,RH1 1SG,https://www.firstincare.co.uk,https://www.cqc.org.uk/location/1-9817102215,redhill,NaN,Worker (A rating),Skilled Worker,2023-03-06,2024-05-15,First In Care Services Ltd,Active,-18771.171211,6.663561e+06
1,Midshires Care Limited,6 College Road,NaN,HA1 1BE,https://www.helpinghands.co.uk,https://www.cqc.org.uk/location/1-3045672865,alcester,Warwickshire,Worker (A rating),Skilled Worker,2022-01-12,2024-05-15,Midshires Care Limited,Active,-37402.903629,6.724579e+06
2,West Sussex Care Services Limited,Units 3 - 4,"20 Northbrook Trading Estate, Northbrook Road",BN14 8PN,NaN,https://www.cqc.org.uk/location/1-9329666383,worthing,West Sussex,Worker (A rating),Skilled Worker,2022-10-17,2024-05-15,West Sussex Care Services Limited,Active,-40456.007643,6.591267e+06
3,Kitec Healthcare Services Limited,"Office B, Elstow Road",Kempston,MK42 9QZ,NaN,https://www.cqc.org.uk/location/1-16437635737,bedford,NaN,Worker (A rating),Skilled Worker,2022-04-12,2024-05-15,Kitec Healthcare Services Limited,Active,-53215.948615,6.821137e+06
4,JSS Homecare Ltd,"Yeovil Innovation Centre, Copse Road","Lufton Trading Estate, Lufton",BA22 8RN,NaN,https://www.cqc.org.uk/location/1-11484415015,yeovil,England,Worker (A rating),Skilled Worker,2023-06-15,2024-05-15,JSS Homecare Ltd,Active,-297648.425588,6.613011e+06
